### Imports

In [63]:
import pandas as pd
import numpy  as np 
import random
import talib
from lightweight_charts import JupyterChart
from colour import Color

from BrainyBackTest.brain import *
from BrainyBackTest.indicator import *
from BrainyBackTest.trade import *

### Data Cleaning

In [64]:
df = pd.read_csv('./0_RawChartData/Crypto_Binance_BTCUSDT.P_5min.csv')

df['time'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])
cols = ['time'] + [col for col in df.columns if col != 'time']
df = df[cols]

df.drop(columns=['DATE', 'TIME', 'TICKVOL'], inplace=True)
df.rename(columns={'time':'time', 'OPEN':'open', 'HIGH':'high', 'LOW':'low', 'CLOSE':'close', 'VOL':'volume', 'SPREAD':'spread'}, inplace=True)

df = df.iloc[:500]
# df = df.head(5000)
df.shape

(500, 7)

### Define the custom indicator

In [65]:
ratioRR  = 10
wantLoss = 1

def My_Custom_OnStart_Function(ind:Indicator ,times:np.array, opens:np.array, highs:np.array, lows:np.array, closes:np.array, spreads:np.array, volumes:np.array):

    ind.set_new_buffer(numBuffer=0, nullValue=0, titleBuffer='ema21', jobBuffer='DrawLine', widthBuffer=1, colorBuffer=Color('green') , windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=1, nullValue=0, titleBuffer='ema55', jobBuffer='DrawLine', widthBuffer=1, colorBuffer=Color('orange'), windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=2, nullValue=0, titleBuffer='ArrowUps', jobBuffer='DrawArrowUps', widthBuffer=1, colorBuffer=Color('green'), windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=3, nullValue=0, titleBuffer='ArrowDns', jobBuffer='DrawArrowDns', widthBuffer=1, colorBuffer=Color('red')  , windowBuffer='SamePanel')
    # ind.set_new_buffer(numBuffer=4, nullValue=0, titleBuffer='Trades', jobBuffer='Signal')


    ind[0, :] = talib.EMA(closes, 21)
    ind[1, :] = talib.EMA(closes, 55)
    

    cross_up   = (ind[0, :-1] <= ind[1, :-1]) & (ind[0, 1:] > ind[1, 1:])
    cross_down = (ind[0, :-1] >= ind[1, :-1]) & (ind[0, 1:] < ind[1, 1:])
    cross_up   = np.concatenate([[False], cross_up])
    cross_down = np.concatenate([[False], cross_down])
    ind[2, cross_up]   = 1111
    ind[3, cross_down] = 1111
#

ind_crossEMA = Indicator("EMA21_X_EMA55", functionOnStart=My_Custom_OnStart_Function)

### Define the custom Brain simulator

In [66]:
BrainMostafaRoohy = Brain(chartData=df, indicators=[ind_crossEMA])

In [67]:
BrainMostafaRoohy.think()

In [68]:
BrainMostafaRoohy.imagine()

In [69]:
BrainMostafaRoohy.speak()

All PnL =  0


### .